### Exploratory analysis on the spotify_tracks dataset

In [1]:
# import required libraries
import pandas as pd
import seaborn as sns
import os

In [2]:
# define a function that is useful for quickly saving a pandas dataframe to an excel
import os
import string
import pandas as pd

def df_xl(df, base_name="data_output"):
    """
    Saves a DataFrame to the 'datasets' folder with incremental alphabetical filenames
    (a–z), then numeric ones (1, 2, 3...).
    Example:
        data_output_a.xlsx, data_output_b.xlsx, ..., data_output_z.xlsx, data_output_1.xlsx, ...
    """

    # Ensure the 'datasets' folder exists
    os.makedirs('datasets', exist_ok=True)

    # List existing files in the folder
    existing_files = [f for f in os.listdir('datasets') if f.startswith(base_name) and f.endswith('.xlsx')]

    # Determine the next filename
    used_suffixes = set()
    for f in existing_files:
        suffix = f[len(base_name):-5]  # remove prefix and .xlsx
        if suffix.startswith("_"):
            suffix = suffix[1:]  # remove leading underscore
        used_suffixes.add(suffix)

    # Generate the next suffix (a–z, then numbers)
    next_suffix = None
    for letter in string.ascii_lowercase:  # a–z
        if letter not in used_suffixes:
            next_suffix = letter
            break
    if next_suffix is None:
        # All letters used — find next number
        numbers = [int(s) for s in used_suffixes if s.isdigit()]
        next_suffix = str(max(numbers) + 1 if numbers else 1)

    # Build the filename and path
    filename = f"{base_name}_{next_suffix}.xlsx"
    output_path = os.path.join(os.getcwd(), "datasets", filename)

    # Save the DataFrame
    df.to_excel(output_path, index=False)
    print(f"DataFrame saved to: {output_path}")


In [3]:
# import the spotify tracks dataset
tracks_df = pd.read_csv("datasets/dataset.csv")

In [3]:
print(tracks_df.head())

   Unnamed: 0                track_id                 artists  \
0           0  5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino   
1           1  4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2           2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3           3  6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4           4  5vjLSffimiIP26QG5WcN2K        Chord Overstreet   

                                          album_name  \
0                                             Comedy   
1                                   Ghost (Acoustic)   
2                                     To Begin Again   
3  Crazy Rich Asians (Original Motion Picture Sou...   
4                                            Hold On   

                   track_name  popularity  duration_ms  explicit  \
0                      Comedy          73       230666     False   
1            Ghost - Acoustic          55       149610     False   
2              To Begin Again          57       210826     False   


In [4]:
# The file has 114k records, so create a sample of the dataset, that I can more easily look at and understand it in excel within VSCode

# take a sample of 5,000 records
tracks_sample_df = tracks_df.sample(n = 5000, random_state = 42) # set the random state for reproducibility

# save to excel in the working folder
output_path = os.path.join(os.getcwd(), "datasets", "dataset_5ksample.xlsx")
tracks_sample_df.to_excel(output_path, index = False)

In [5]:
# understand dataset structure
print(tracks_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

In [6]:
# Make sure that the field track_id is a unique primary index

# how many records in the dataset?
print(tracks_df.shape) # 114,000

# how many unique track_ids are there
print(tracks_df['track_id'].nunique())  # 89,741 (so a lot of duplication at track_id). investigate this next


(114000, 21)
89741


In [7]:

# create a dataframe showing the number of occurrences of each track_id
track_id_agg_df = tracks_df['track_id'].value_counts().reset_index()
track_id_agg_df.columns = ['track_id', 'num_occ']
print(track_id_agg_df.head())

                 track_id  num_occ
0  6S3JlDAGk3uu3NtZbPnuhS        9
1  2Ey6v4Sekh3Z0RUSISRosD        8
2  2kkvB3RNRzwjFdGhaUA0tz        8
3  2vU6bm5hVF2idVknGzqyPL        7
4  5ZsAhuQ24mWHiduaxJqnhW        7


In [8]:
# aggregate this further to show how many track_ids have how many occurrences
track_id_agg_df_agg  = track_id_agg_df['num_occ'].value_counts().reset_index()
track_id_agg_df_agg.columns = ['num_occ', 'num_track_id']
track_id_agg_df_agg.sort_values('num_occ', ascending = True)
track_id_agg_df_agg['cumsum_num_track_id'] = track_id_agg_df_agg['num_track_id'].cumsum()
track_id_agg_df_agg['prop'] = track_id_agg_df_agg['num_track_id'] / track_id_agg_df_agg['num_track_id'].sum()
track_id_agg_df_agg['cumsum_prop'] = track_id_agg_df_agg['prop'].cumsum()
print(track_id_agg_df_agg)

   num_occ  num_track_id  cumsum_num_track_id      prop  cumsum_prop
0        1         73100                73100  0.814566     0.814566
1        2         11712                84812  0.130509     0.945075
2        3          2984                87796  0.033251     0.978327
3        4          1372                89168  0.015288     0.993615
4        5           431                89599  0.004803     0.998418
5        6           117                89716  0.001304     0.999721
6        7            22                89738  0.000245     0.999967
7        8             2                89740  0.000022     0.999989
8        9             1                89741  0.000011     1.000000


In [9]:
# only 81% of the track ids are unique, but 99% have 4 or less occurences
# take a sample of the duplicates to begin figuring out some strategies for handling this issue

In [10]:
# create a random sample  of duplicate track_ids
track_id_dup = track_id_agg_df[track_id_agg_df['num_occ'] > 1]
print(track_id_dup.head(10))
# sort it to make sure there's no 1s
track_id_dup.sort_values('num_occ', ascending = True)
# all good

                 track_id  num_occ
0  6S3JlDAGk3uu3NtZbPnuhS        9
1  2Ey6v4Sekh3Z0RUSISRosD        8
2  2kkvB3RNRzwjFdGhaUA0tz        8
3  2vU6bm5hVF2idVknGzqyPL        7
4  5ZsAhuQ24mWHiduaxJqnhW        7
5  4XYieGKSlJlHpzB3bl6WMP        7
6  3dJjsWCy6ZwMfw5NbD226G        7
7  0YLSjVxSb5FT1Bo8Tnxr8j        7
8  4aqS25F3ywJ9TGnNkOqilC        7
9  0e5LcankE0UyJUuCoq1uH2        7


,track_id,num_occ
16624,27nGU2v3syK7aU3AVY2vUO,2
16625,2TgTGJyiWf1ptW5g3QG938,2
16626,63vjnB6EeQuVf64zLxIo90,2
16627,0sSjIvTvd6fUSZZ5rnTPDW,2
16628,2zg3iJW4fK7KZgHOvJU67z,2
...,...,...
17,08kTa3SL9sV6Iy8KLKtGql,7
22,4GPQDyw9hC1DiZVh0ouDVL,7
1,2Ey6v4Sekh3Z0RUSISRosD,8
2,2kkvB3RNRzwjFdGhaUA0tz,8


In [11]:
# take 20 at random
dup_tracks_sample = track_id_dup.sample(n = 20, random_state = 58) # set the random state for reproducibility
print(dup_tracks_sample)

                     track_id  num_occ
1976   3McXs3tqyP5VwBg4FdiILD        3
1844   2GqmufsIytFAXE9sxz4m4S        4
12003  3rGol3Ivk8X9fvzKL0U8ih        2
350    3AedlLCGKNhQdhtLzObIbR        5
61     1ycq2btx7kf6bXdWILZhXa        6
5952   22sQUmLhT8umlEhQzDrzfJ        2
15270  4XSpyx2fV1c9Iq9TYY2zXk        2
16067  4SeZpWSgq1c8a0MY3iVy1f        2
8637   6FOZMPsV4o3dMj7juFrXMT        2
10468  0zqBZqm5czQ3A4EoSdKFHj        2
4378   1ErVAnzjNvyXSodWbOTYNu        3
7208   46jni4B6gnRB5EbjIEnsf3        2
10030  7lUB18zW8flnzCpHmIA0lL        2
6656   46RvocxUkMUhCtDxk1BUyT        2
7947   2YNgcIiD73XsXFNM3UuxlM        2
666    5ln5yQdUywVbf8HhFsOcd6        4
15379  1DIGB5kZKiHnEb2uWxzjzY        2
3079   0E9ZjEAyAwOXZ7wJC0PD33        3
13953  3qNERYCjkBFyhYUNh4Hl9K        2
920    4jJ3rcbPevOdXr6hFIVW45        4


In [12]:
tracks_df_dup = tracks_df[tracks_df['track_id'].isin(dup_tracks_sample['track_id'])] 
tracks_df_dup.sort_values(by = 'track_id', ascending = True)

print(tracks_df_dup)


        Unnamed: 0                track_id                        artists  \
5867          5867  22sQUmLhT8umlEhQzDrzfJ                           Reol   
9331          9331  6FOZMPsV4o3dMj7juFrXMT                    Juliano Son   
20259        20259  5ln5yQdUywVbf8HhFsOcd6                         Halsey   
21143        21143  3McXs3tqyP5VwBg4FdiILD        Vybz Kartel;Arif Cooper   
23143        23143  46jni4B6gnRB5EbjIEnsf3                      BLOND:ISH   
25277        25277  46jni4B6gnRB5EbjIEnsf3                      BLOND:ISH   
26165        26165  7lUB18zW8flnzCpHmIA0lL                      Jon Sarta   
28105        28105  46RvocxUkMUhCtDxk1BUyT            ILLENIUM;Dana Salah   
29060        29060  46RvocxUkMUhCtDxk1BUyT            ILLENIUM;Dana Salah   
30518        30518  0E9ZjEAyAwOXZ7wJC0PD33         Dynoro;Gigi D'Agostino   
31025        31025  5ln5yQdUywVbf8HhFsOcd6                         Halsey   
31118        31118  0zqBZqm5czQ3A4EoSdKFHj  Alan Walker;Benjamin Ingrosso   

In [13]:
# that sorting didn't work, since they're alphanumeric. try again
import re

# Extract leading number
tracks_df_dup['numeric_prefix'] = tracks_df_dup['track_id'].str.extract(r'^(\d+)').astype(int)

# Sort by numeric prefix first, then full string
tracks_df_dup.sort_values(by=['numeric_prefix', 'track_id'], inplace=True)

# Optional: drop helper column
tracks_df_dup.drop(columns='numeric_prefix', inplace=True)

C:\Users\Anne\AppData\Local\Temp\ipykernel_12916\2776651592.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df_dup['numeric_prefix'] = tracks_df_dup['track_id'].str.extract(r'^(\d+)').astype(int)
C:\Users\Anne\AppData\Local\Temp\ipykernel_12916\2776651592.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks_df_dup.sort_values(by=['numeric_prefix', 'track_id'], inplace=True)
C:\Users\Anne\AppData\Local\Temp\ipykernel_12916\2776651592.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

In [14]:
print(tracks_df_dup)

        Unnamed: 0                track_id                        artists  \
30518        30518  0E9ZjEAyAwOXZ7wJC0PD33         Dynoro;Gigi D'Agostino   
31572        31572  0E9ZjEAyAwOXZ7wJC0PD33         Dynoro;Gigi D'Agostino   
53361        53361  0E9ZjEAyAwOXZ7wJC0PD33         Dynoro;Gigi D'Agostino   
31118        31118  0zqBZqm5czQ3A4EoSdKFHj  Alan Walker;Benjamin Ingrosso   
53114        53114  0zqBZqm5czQ3A4EoSdKFHj  Alan Walker;Benjamin Ingrosso   
86919        86919  1DIGB5kZKiHnEb2uWxzjzY       Big D and the Kids Table   
100373      100373  1DIGB5kZKiHnEb2uWxzjzY       Big D and the Kids Table   
67784        67784  1ErVAnzjNvyXSodWbOTYNu                 Greeicy;Anitta   
88542        88542  1ErVAnzjNvyXSodWbOTYNu                 Greeicy;Anitta   
89493        89493  1ErVAnzjNvyXSodWbOTYNu                 Greeicy;Anitta   
55435        55435  1ycq2btx7kf6bXdWILZhXa                     Anupam Roy   
56134        56134  1ycq2btx7kf6bXdWILZhXa                     Anupam Roy   

In [15]:
# save to excel so that I can see it properly
output_path2 = os.path.join(os.getcwd(), "datasets", "dup_sample.xlsx")
tracks_df_dup.to_excel(output_path2, index = False)

In [16]:
# ok the issue is that there aren't duplicate tracks as such
# if a track has multiple genres it has a record for the first genre, a record for the second genre etc.
# one solution to fix this would be to make the genre column 'wide' 
# i.e. a column for pop that has a 1 or 0 if the track has the pop 'genre', a column 'rock' (1 or 0 if track is rock) etc.

# what are the distinct values on the track_genre field?
genres = tracks_df['track_genre'].value_counts().reset_index()

In [17]:
output_path3 = os.path.join(os.getcwd(), "datasets", "genres.xlsx")
genres.to_excel(output_path3, index = False)

In [18]:
# ok, it appears there are 114 different genres, with 1000 tracks for each, this means that there will be a bias to mitigate against later

In [19]:
# take the music of a specific artist to see if there is anything else to watch out for

cb_df = tracks_df[
    tracks_df['artists'].str.contains('chris', case=False, na=False) &
    tracks_df['artists'].str.contains('brown', case=False, na=False)
]

output_path4 = os.path.join(os.getcwd(), "datasets", "cb.xlsx")
cb_df.to_excel(output_path4, index = False)

In [20]:
# yes, the song 'Algo me gusta de ti' is a duplicate because it appears on multiple different albums.
# The only thing that differs for it is the popularity and the genre, so maybe the most popular version could be taken to simplify things

In [21]:
# make the genre column wide

# first replace the hyphens with underscores
tracks_df['track_genre_cln']  = tracks_df['track_genre'].str.replace('-','_', regex = False)

# check that it worked as desired
test = tracks_df[['track_genre', 'track_genre_cln']].drop_duplicates()
print(test)

# all good

        track_genre track_genre_cln
0          acoustic        acoustic
1000       afrobeat        afrobeat
2000       alt-rock        alt_rock
3000    alternative     alternative
4000        ambient         ambient
...             ...             ...
109000       techno          techno
110000       trance          trance
111000     trip-hop        trip_hop
112000      turkish         turkish
113000  world-music     world_music

[114 rows x 2 columns]


In [23]:
# convert the cln column to a wide
tracks_df['track_genre_cln'] = tracks_df['track_genre_cln'].astype('category')
print(tracks_df.info()) # worked as desired

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   Unnamed: 0        114000 non-null  int64   
 1   track_id          114000 non-null  object  
 2   artists           113999 non-null  object  
 3   album_name        113999 non-null  object  
 4   track_name        113999 non-null  object  
 5   popularity        114000 non-null  int64   
 6   duration_ms       114000 non-null  int64   
 7   explicit          114000 non-null  bool    
 8   danceability      114000 non-null  float64 
 9   energy            114000 non-null  float64 
 10  key               114000 non-null  int64   
 11  loudness          114000 non-null  float64 
 12  mode              114000 non-null  int64   
 13  speechiness       114000 non-null  float64 
 14  acousticness      114000 non-null  float64 
 15  instrumentalness  114000 non-null  float64 
 16  li

In [ ]:
# I need to make sure that the first 20 columns (all except the track_genre columns) are a unique primary key
cols = tracks_df.columns.tolist()
print(cols)

column_names = ['track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms'
                , 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness'
                , 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']

# just show these columns in the dataframe
tracks_df_subset = tracks_df[column_names]

# remove the duplicates
tracks_df_subset_dedup = tracks_df_subset.drop_duplicates()

# now check whether there are any duplicate track_ids
test = tracks_df_subset_dedup['track_id'].value_counts().reset_index()
print(test)
test.columns = ['track_id', 'num_occ']
test2 = test['num_occ'].value_counts().reset_index()
print(test2)
# still 720 with a duplicate
# isolate them

test3 = test[test['num_occ'] == 2]
print(test3)


test4 = list(test3['track_id'])
print(test4)


test5 = tracks_df_subset_dedup[tracks_df_subset_dedup['track_id'].isin(test4)]
print(test5)

# check whether there are any duplicates in the dataframe
#duplicates_exist = tracks_df_subset.duplicated().any()
#print("Any duplicates:", duplicates_exist)

# gather the duplicated rows for analysis
#duplicated_rows = tracks_df_subset[tracks_df_subset.duplicated(keep=False)]


# take a sample of the duplicated rows as there seem to be a lot
#duplicated_rows_samp = duplicated_rows.sample(n = 5000, random_state = 38) 

#output them for easy review
output_path6 = os.path.join(os.getcwd(), "datasets", "duplicates5.xlsx")
test5.to_excel(output_path6, index = False)


# there are also duplicates on the popularity column


['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'track_genre', 'track_genre_cln']
                     track_id  count
0      2ji53kdnw4UtV6PIt94MLq      2
1      30hKEQLgr25oI91btWLvCi      2
2      25ysC9uw0bpwgcxOhnrwjk      2
3      3TNUCQeXAALQrTw7ANSEoB      2
4      7nudwi49ZHlgt6AtNNgVnJ      2
...                       ...    ...
89736  5wMkhfu3oYYmFimOGzeFt2      1
89737  64BdW5aHuB3C0QP3qjBJVi      1
89738  1pBc7W7UQacUeQ6kBVlN9f      1
89739  39So2wAM6l3XKWDf5XjiIz      1
89740  01MVOl9KtVTNfFiBU9I7dc      1

[89741 rows x 2 columns]
   num_occ  count
0        1  89021
1        2    720
                   track_id  num_occ
0    2ji53kdnw4UtV6PIt94MLq        2
1    30hKEQLgr25oI91btWLvCi        2
2    25ysC9uw0bpwgcxOhnrwjk        2
3    3TNUCQeXAALQrTw7ANSEoB      

In [ ]:
# a list of identified duplicates
dups = ['7tNzd03v3T5aU7P1UpeuYF','7Fo51x6cURFo9jCzcv7WOL','4WRoLfCYJ4sLMhXgQJnI2N','7ITEWyZlNF2bpDPsHan8Ea','7EkWXAI1wn8Ii883ecd9xr',
        '4Q2pNhiQ5KtzuXd0GIbRiz','5psIcronqyDyYWBVFNSyVh','5X1VGqaNrpXC8HBPEA4uDT','59mrqUmhpmcfUns8BKkV30','2jdAk8ATWIL3dwT47XpRfu',
        '7aonAWn0J0AJ47ZU9WHCXC','2KH9tWMNpKRIfEJlx8FJbk','5oDkgd1jpgrT0wcHwtsSQq','2SFhJe79faMbb5fgEFJGg0','4wSMi5urWxpBPbnWqfI1Ht',
        '0PF7SVd93G602xdi8SVyEz','1yEBMvOYbhJJTX7x3iW41O','3MADBbqQFspWXZrTytVLfO','7aQT5OB4zEgyyxfRDbOBIt']

# filter for them
dups2 = tracks_df_subset[tracks_df_subset['track_id'].isin(dups)]

# output to excel for a quick look
output_path6 = os.path.join(os.getcwd(), "datasets", "duplicates2.xlsx")
dups2.to_excel(output_path6, index = False)

# ok, they're all duplicates there


In [28]:
# compare that with a handful of them in the original data frame
dups3 = ['0PF7SVd93G602xdi8SVyEz','1yEBMvOYbhJJTX7x3iW41O','2jdAk8ATWIL3dwT47XpRfu']
# filter for them
dups4 = tracks_df[tracks_df['track_id'].isin(dups)]

# output to excel for a quick look
output_path7 = os.path.join(os.getcwd(), "datasets", "duplicates3.xlsx")
dups4.to_excel(output_path7, index = False)

In [ ]:
tracks_df['value'] = 1  # This will be used as the value for pivot

# Pivot only track_id + genre
tracks_df2 = tracks_df.pivot_table(
    index='track_id',
    columns='track_genre_cln',
    values='value',
    aggfunc='max',
    fill_value=0
).reset_index()

# Merge with metadata
#df_wide = df_metadata.merge(df_pivot, on='track_id', how='left')


C:\Users\Anne\AppData\Local\Temp\ipykernel_2900\269178420.py:4: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tracks_df2 = tracks_df.pivot_table(


In [ ]:
output_path5 = os.path.join(os.getcwd(), "datasets", "tracksdf2.xlsx")
tracks_df2.to_excel(output_path5, index = False)

In [ ]:
# look at one of the popularity duplicates in the original dataset
testa = tracks_df[tracks_df['track_id'] == '00YwP3wJWiG8IxAA7OS9lo']

output_path6 = os.path.join(os.getcwd(), "datasets", "testa.xlsx")
testa.to_excel(output_path6, index = False)

# interestingly, it is the same song but has different genres


In [ ]:
# solution
# make one dataset of unique track_id, trackname, artist album etc. EXCEPT the genre, and just take the max of popularity
# make a second dataset of unique track_id, trackname, artist album, all the unique genres linked to it, but making sure to leave out the popularity part
# make the second dataset wide, then left join it onto the first
# then it'll be clean and red for exploratory analysis



In [32]:
# make the first dataset
tracks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 22 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

In [4]:
# make the first dataset
tracks_df_build1 = tracks_df[['track_id', 'artists', 'album_name', 'track_name', 'popularity', 'duration_ms'
                            , 'explicit', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness'
                            , 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
tracks_df_build1.info() # 1140k records


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   track_id          114000 non-null  object 
 1   artists           113999 non-null  object 
 2   album_name        113999 non-null  object 
 3   track_name        113999 non-null  object 
 4   popularity        114000 non-null  int64  
 5   duration_ms       114000 non-null  int64  
 6   explicit          114000 non-null  bool   
 7   danceability      114000 non-null  float64
 8   energy            114000 non-null  float64
 9   key               114000 non-null  int64  
 10  loudness          114000 non-null  float64
 11  mode              114000 non-null  int64  
 12  speechiness       114000 non-null  float64
 13  acousticness      114000 non-null  float64
 14  instrumentalness  114000 non-null  float64
 15  liveness          114000 non-null  float64
 16  valence           11

In [ ]:
# drop the duplicates
tracks_df_build1a = tracks_df_build1.drop_duplicates()
tracks_df_build1a.info() # 90k records

<class 'pandas.core.frame.DataFrame'>
Index: 90461 entries, 0 to 113999
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          90461 non-null  object 
 1   artists           90460 non-null  object 
 2   album_name        90460 non-null  object 
 3   track_name        90460 non-null  object 
 4   popularity        90461 non-null  int64  
 5   duration_ms       90461 non-null  int64  
 6   explicit          90461 non-null  bool   
 7   danceability      90461 non-null  float64
 8   energy            90461 non-null  float64
 9   key               90461 non-null  int64  
 10  loudness          90461 non-null  float64
 11  mode              90461 non-null  int64  
 12  speechiness       90461 non-null  float64
 13  acousticness      90461 non-null  float64
 14  instrumentalness  90461 non-null  float64
 15  liveness          90461 non-null  float64
 16  valence           90461 non-null  float64
 1

In [5]:
# now just take all values and the maximum populatiry value

# Define which column to aggregate
agg_col = 'popularity'

# Group by all other columns
group_cols = [c for c in tracks_df_build1a.columns if c != agg_col]

# Aggregate
tracks_df_build1b = tracks_df_build1a.groupby(group_cols, as_index=False)[agg_col].max()

NameError: name 'tracks_df_build1a' is not defined

In [ ]:
# output and make sure I'm happy with it
tracks_df_build1b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89740 entries, 0 to 89739
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          89740 non-null  object 
 1   artists           89740 non-null  object 
 2   album_name        89740 non-null  object 
 3   track_name        89740 non-null  object 
 4   duration_ms       89740 non-null  int64  
 5   explicit          89740 non-null  bool   
 6   danceability      89740 non-null  float64
 7   energy            89740 non-null  float64
 8   key               89740 non-null  int64  
 9   loudness          89740 non-null  float64
 10  mode              89740 non-null  int64  
 11  speechiness       89740 non-null  float64
 12  acousticness      89740 non-null  float64
 13  instrumentalness  89740 non-null  float64
 14  liveness          89740 non-null  float64
 15  valence           89740 non-null  float64
 16  tempo             89740 non-null  float6

In [39]:
# it has one less record than the main dataset, investigate why
missing_in_df1 = tracks_df_build1b[~tracks_df_build1b['track_id'].isin(tracks_df['track_id'])]
missing_in_df2 = tracks_df[~tracks_df['track_id'].isin(tracks_df_build1b['track_id'])]
print(missing_in_df1)
print(missing_in_df2)

Empty DataFrame
Columns: [track_id, artists, album_name, track_name, duration_ms, explicit, danceability, energy, key, loudness, mode, speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature, popularity]
Index: []
       Unnamed: 0                track_id artists album_name track_name  \
65900       65900  1kR4gIb7nGxHPI3D2ifs59     NaN        NaN        NaN   

       popularity  duration_ms  explicit  danceability  energy  ...  mode  \
65900           0            0     False         0.501   0.583  ...     0   

       speechiness  acousticness  instrumentalness  liveness  valence  \
65900       0.0605          0.69           0.00396    0.0747    0.734   

         tempo  time_signature  track_genre  track_genre_cln  
65900  138.391               4        k-pop            k_pop  

[1 rows x 22 columns]


In [ ]:
testb = tracks_df[tracks_df['track_id'] == '1kR4gIb7nGxHPI3D2ifs59']

output_path7 = os.path.join(os.getcwd(), "datasets", "testb.xlsx")
testb.to_excel(output_path7, index = False)

# it's just a solitary k-pop song without a trackname, artist, album or popularity value. So just exclude it from the dataset


In [ ]:
tracks_df_build1b['track_id'].nunique()
# all good

89740

In [43]:
# moving on to the second dataset
tracks_df_build2 = tracks_df[['track_id', 'artists', 'album_name', 'track_name', 'track_genre_cln']]
tracks_df_build2.info() # 114k records

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   track_id         114000 non-null  object
 1   artists          113999 non-null  object
 2   album_name       113999 non-null  object
 3   track_name       113999 non-null  object
 4   track_genre_cln  114000 non-null  object
dtypes: object(5)
memory usage: 4.3+ MB


In [47]:
tracks_df_build2a = tracks_df_build2.drop_duplicates()
tracks_df_build2a.info()

<class 'pandas.core.frame.DataFrame'>
Index: 113550 entries, 0 to 113999
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   track_id         113550 non-null  object
 1   artists          113549 non-null  object
 2   album_name       113549 non-null  object
 3   track_name       113549 non-null  object
 4   track_genre_cln  113550 non-null  object
dtypes: object(5)
memory usage: 5.2+ MB


In [46]:
check_a = tracks_df[tracks_df['track_id'].isin(['6hNjGPE8S1No2Evu4aPLjL', '1gQ0W0XKnOUpmcD5Vzs8e7', '2R3LQndjqKf8CIuPTSiI8p', '476V2d6iA2tWXgQboKmTtA', '5zTirec9tIsSVNgBSUmFxU', 
                                                '5r0x87jwTbkVjA3FQNOykz', '3RXAcz7Sa6JDZSxcH1EEQ6', '6aGP0AAF8iLyoGdLRpuVoZ', '7fpym9dJ7HFNgOxk2x8iMb', '2syQ0f4SAQdzvlAewPl1DJ', 
                                                ])]
df_xl(check_a)

DataFrame saved to: c:\Users\Anne\OneDrive\Desktop\Files\_VSC_repositories\spotify_tracks\datasets\data_output_a.xlsx


In [50]:
# it looks like all the tracks with a different performance value, also have different values for the genre.
# it makes sense in the interest of not excluding information, to keep all genres

# so, make the dataset wide
# Create dummy variables (one-hot encoding)
dummies = pd.get_dummies(tracks_df_build2a['track_genre_cln'])

# Concatenate the dummy columns to the original DataFrame
df_wide = pd.concat([tracks_df_build2a.drop(columns=['track_genre_cln']), dummies], axis=1)

# Aggregate to remove duplicates — sum ensures 1 if category appears
tracks_df_build2b = df_wide.groupby(['track_id', 'artists', 'album_name', 'track_name'], as_index=False).sum()

In [51]:
tracks_df_build2b.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89740 entries, 0 to 89739
Columns: 118 entries, track_id to world_music
dtypes: int64(114), object(4)
memory usage: 80.8+ MB


In [58]:
# left join dataset 2 onto dataset 1 and do some sampling to test the popularity and category cleanings

tracks_df_cln = tracks_df_build1b.merge(tracks_df_build2b, on = ['track_id', 'artists', 'album_name','track_name'], how = 'left')
tracks_df_cln.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89740 entries, 0 to 89739
Columns: 133 entries, track_id to world_music
dtypes: bool(1), float64(9), int64(119), object(4)
memory usage: 90.5+ MB


In [59]:
# take a sample of it and clash against the original dataframe
tracks_df_cln_sample = tracks_df_cln[tracks_df_cln['track_id'].isin(['6hNjGPE8S1No2Evu4aPLjL', '1gQ0W0XKnOUpmcD5Vzs8e7', '2R3LQndjqKf8CIuPTSiI8p', '476V2d6iA2tWXgQboKmTtA', '5zTirec9tIsSVNgBSUmFxU', 
                                                '5r0x87jwTbkVjA3FQNOykz', '3RXAcz7Sa6JDZSxcH1EEQ6', '6aGP0AAF8iLyoGdLRpuVoZ', '7fpym9dJ7HFNgOxk2x8iMb', '2syQ0f4SAQdzvlAewPl1DJ', 
                                                ])]
df_xl(tracks_df_cln_sample)

tracks_df_sample2 = tracks_df[tracks_df['track_id'].isin(['6hNjGPE8S1No2Evu4aPLjL', '1gQ0W0XKnOUpmcD5Vzs8e7', '2R3LQndjqKf8CIuPTSiI8p', '476V2d6iA2tWXgQboKmTtA', '5zTirec9tIsSVNgBSUmFxU', 
                                                '5r0x87jwTbkVjA3FQNOykz', '3RXAcz7Sa6JDZSxcH1EEQ6', '6aGP0AAF8iLyoGdLRpuVoZ', '7fpym9dJ7HFNgOxk2x8iMb', '2syQ0f4SAQdzvlAewPl1DJ', 
                                                ])]

df_xl(tracks_df_sample2)

DataFrame saved to: c:\Users\Anne\OneDrive\Desktop\Files\_VSC_repositories\spotify_tracks\datasets\data_output_b.xlsx
DataFrame saved to: c:\Users\Anne\OneDrive\Desktop\Files\_VSC_repositories\spotify_tracks\datasets\data_output_c.xlsx


In [ ]:
# perfectly as desired from sampling

In [ ]:
# but there is also the issue of duplicate tracks across albums
check = tracks_df_cln[tracks_df_cln['track_name']=='Algo Me Gusta De Ti']
df_xl(check)

DataFrame saved to: c:\Users\Anne\OneDrive\Desktop\Files\_VSC_repositories\spotify_tracks\datasets\data_output_e.xlsx


In [ ]:
# explore a solution for this
# how much duplication is there across track_name, artists, duration_ms
